Ship direction can be explained recursively

\begin{equation}
    \begin{cases}
        \eta_k &= \eta_{k - 1} + \xi_k, \\
        \eta_0 &= 0,
    \end{cases}
    \Rightarrow
    \eta_k = \sum\limits_{i = 1}^k \xi_k
\end{equation}

where $\xi_k \sim N\left( 0, \frac{1}{16} \right)$,
and thus $\eta_k \sim N\left(0, \frac{k}{16} \right)$.

Thus, position of the ship is a process that depends on the $\eta$

\begin{equation}
    \begin{cases}
        \zeta^x_k &= \zeta^x_{k - 1} - 15 \cdot \cos{\eta_k}, \\
        \zeta^y_k &= \zeta^y_{k - 1} + 15 \cdot \sin{\eta_k}, \\
        \zeta^x_0 &= 500, \\
        \zeta^y_0 &= 0,
    \end{cases}
    \Rightarrow
    \begin{cases}
        \zeta^x_k &= 500 - 15 \cdot \sum\limits_{i = 1}^k \cos{\xi_i}, \\
        \zeta^y_k &= 15 \cdot \sum\limits_{i = 1}^k \sin{\xi_i},
    \end{cases}
\end{equation}

The task is to plot distributions of $\pmb{\zeta}_{12}$ and $\pmb{\zeta}_{24}$.

It's hard to calculate a distribution of $\zeta^x_k$,
because this will need to calculate a $k$-multiple integral
to have a convolution of probabilities of cosines of dependent normally distributed random variables.
They are dependent, because $\eta_k$ depends on $\eta_i$ through $\xi_i, i < k$.

The idea is to calculate the first "ring" of probabilities (after one hour).
Then, for each point of the ring, calculate "second-order" rings, and so on.
Given a grid with specified density, we can go through each point,
and, for each point of a circle with radius of $15$ nautical miles,
calculate a probability of the ship to be there in one hour,
and multiply it by probability of the ship to be in current point.

Cumulative distribution function of $\sin{\xi_k}$ is

\begin{equation}
    F_{\sin{\xi_k}}\left( x \right) = P\left( \sin{\xi_k} \le x \right)
\end{equation}

Sine is a periodic function, so we cannot just apply $arcsin$ to both sides of the inequality.
Can we?

Let's calculate what we can lose by this trick

\begin{equation}
    P\left( \left| \xi_k \right| \le \pi \right) = 2 \cdot P\left( \xi_k \le -\pi \right) = 2 \cdot F_{\xi_k}\left( -\pi \right)
\end{equation}

In [1]:
from numpy import pi
from scipy.stats import norm

tail_weight = 2 * norm.cdf(-2 * pi, scale=1/4)
print(f"Tail weight is {tail_weight * 100:e}%")

Tail weight is 2.182448e-137%


We can see that if we assume that after one hour angle may not change for more than $\pi$ in any direction,
we don't lose anything,
because the probability $\sim 10^{-137}$ is very small.

Though, we want to sum several random variables under this assumption.
Let $\tilde{p}$ be the probability of an angle to change for more than $\pi$ after an hour in any direction.
Then, the probability of staying in this bound is $1 - \tilde{p}$.
The probability that neither change will overflow the bound within $k$ hours is $\left( 1 - \tilde{p} \right)^k$.
Let's apply binomial theorem to calculate a probability that at least one direction change will overflow specified boundary

\begin{equation}
    1 - \left( 1 - \tilde{p} \right)^n
    = 1 - \sum\limits_{k=0}^n {n \choose k} \cdot \left( -\tilde{p} \right)^k
\end{equation}

In [2]:
from numpy import pi
from scipy.stats import norm
from scipy.special import comb

final_tail = sum(comb(24, i) * (-2 * norm.cdf(-pi, scale=1/4)) ** i for i in range(1, 25))
print(f"Weight of tail after summation of chain of 24 random variables is {-final_tail * 100:e}%")

Weight of tail after summation of chain of 24 random variables is 7.757390e-33%


The tail grows fast, but it still very tiny and we can ignore it.